<a href="https://colab.research.google.com/github/dangoldmann/TP-Netflix/blob/main/TP_Netflix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Trabajo Práctico Netflix**
### *Septiembre 2022*

Descargar el archivo que contiene el historial de Netflix. Pueden descargar el suyo propio entrando a este link. A resolver de a grupos de 2 o 3 integrantes.

Ignorar los siguientes dispositivos: 'Sony PS4', 'Netflix Windows App - Cadmium Windows Mobile', 'Apple iPhone 6s', 'Apple iPhone 7 Plus (CDMA)', 'Chrome PC (Cadmium)', 'Samsung 2015 NT14M DTV Smart TV', 'iPhone 5S GSM', 'Google Chromecast V3 Streaming Stick'.

- Contabilizar el dispositivo con el que más títulos se reprodujeron.
- Encontrar la película con más ocurrencias y la película con mayor duración total.
- Generar un TOP “N” de series vistas y graficar.
 - Por tiempo visto
 - Por ocurrencias
- Visualizar una semana de reproducciones genérica, en función a las reproducciones por día y por horario de todos los años.
Contabilizar la cantidad de títulos vistos por año, graficar su variación a través de los años.
Total de días de todos los títulos reproducidos.

### **Bonus**

- Identificar cuál fue la película de Star Wars más veces vista por duración.
- Utilizando la API de IMDb (u otra):
 - Mostrar el póster de las películas más vistas, y de las series.
 - Encontrar el género de series y el de películas más ocurrente.
- Encontrar la serie que más rápido se terminó de ver.


Formato de entrega: Link a GitHub conteniendo el Ipython Notebook con su output guardado. No se corrigen Notebooks sin output generado ni entregas que no sean links a GitHub directamente. Se va a evaluar formato del Notebook, celdas de texto, prolijidad del código y de los resultados generados.




Importando librerías

In [133]:
import numpy as np
import pandas as pd

Guardando el dataframe

In [201]:
df = pd.read_csv("./sample_data/NetflixHistory.csv")

#### Eliminando filas no deseadas del dataframe

Eliminando las visualizaciones de los dispositivos no deseados

In [203]:
dispositivosIgnorados = ["Sony PS4", "Netflix Windows App - Cadmium Windows Mobile", "Apple iPhone 6s", "Apple iPhone 7 Plus (CDMA)", "Chrome PC (Cadmium)", "Samsung 2015 NT14M DTV Smart TV", "iPhone 5S GSM", "Google Chromecast V3 Streaming Stick"]

for i in dispositivosIgnorados:
  df = df[df["Device Type"] != i]

Eliminando los hooks, trailers y recaps

In [205]:
video_types = df.groupby(["Supplemental Video Type"]).size()
video_types

Supplemental Video Type
HOOK              49
RECAP              4
TEASER_TRAILER     3
TRAILER           34
dtype: int64

In [206]:
videoTypesIgnorados = ["HOOK", "RECAP", "TEASER_TRAILER", "TRAILER"]

for i in videoTypesIgnorados:
  df = df[df["Supplemental Video Type"] != i]

Eliminando columnas no utilizadas

In [208]:
columnasIgnoradas = ["Supplemental Video Type", "Attributes", "Country"]

for i in columnasIgnoradas:
  df = df.drop(columns = i)

#### Modificando las columnas

Crear una nueva columna que contenga si la visualización corresponde a una película o una serie

In [210]:
def defineContentType(input):
  if "Temporada" in input:
    return "Serie"
  else:
    return "Pelicula"

In [211]:
df["Content Type"] = df["Title"].apply(defineContentType)

Cambiando los valores de las duraciones a segundos

In [216]:
def changeDurationToSec(input):
  try:
    horas = int(input[0:2])
    minutos = int(input[3:5])
    segundos = int(input[6:8])

    return horas * 60 * 60 + minutos * 60 + segundos
  except:
    return input

In [225]:
df["Duration"] = df["Duration"].apply(changeDurationToSec)

#### Contabilizar el dispositivo con el que más títulos se reprodujeron

In [229]:
dispositivos = df.groupby(["Device Type"]).size()
max_cant = max(dispositivos)
dispositivos = dispositivos.reset_index()

In [230]:
dispositivoMasUtilizado = dispositivos.loc[dispositivos[0]==max_cant]
dispositivoMasUtilizado

,Device Type,0
1,Apple Apple TV 4 Apple TV,3439


#### Encontrar la película con más ocurrencias y la película con mayor duración total

In [251]:
df_peliculas = df[df["Content Type"] == "Pelicula"]

Mayor ocurrencias

In [256]:
peliculas = df_peliculas.groupby(["Title"]).size().reset_index()
max_pelicula = max(peliculas[0])
peliMasOcurrida = peliculas.loc[peliculas[0]==max_pelicula]
peliMasOcurrida

,Title,0
123,El código enigma,7
226,Star Wars: El Imperio contraataca,7


Mayor duración

In [257]:
durations = df_peliculas.groupby(['Title', 'Duration']).size().reset_index()
max_duration = max(durations["Duration"])
peliMasDuracion = durations.loc[durations["Duration"] == max_duration]
peliMasDuracion

,Title,Duration,0
175,El club de la pelea,8411,1


## Mostrando el dataframe

In [228]:
original_df = pd.read_csv("./sample_data/NetflixHistory.csv")

In [226]:
df

,Unnamed: 0,Profile Name,Start Time,Duration,Title,Device Type,Bookmark,Latest Bookmark,Content Type
0,0,Chona,2022-08-21 19:54:53,174,Community: Temporada 3: Teoría del caos correc...,Apple Apple TV 4 Apple TV,00:02:55,00:02:55,Serie
1,1,Chona,2022-08-21 19:33:17,1293,Community: Temporada 3: Ecología competitiva (...,Apple Apple TV 4 Apple TV,00:21:33,00:21:33,Serie
2,2,Chona,2022-08-21 19:11:40,1288,Community: Temporada 3: Geografía del conflict...,Apple Apple TV 4 Apple TV,00:21:28,00:21:28,Serie
3,3,Chona,2022-08-21 18:50:27,1241,Community: Temporada 2: Intermediate Documenta...,Apple Apple TV 4 Apple TV,00:21:02,00:21:02,Serie
4,4,Chona,2022-08-21 18:16:34,1280,Community: Temporada 3: El documental (Capítul...,Apple Apple TV 4 Apple TV,00:21:21,00:21:21,Serie
...,...,...,...,...,...,...,...,...,...
12992,12992,Chona,2015-01-08 19:21:21,249,Lost: Temporada 1: Éxodo: Parte 1 (Capítulo 23),Apple TV 3 (revised),00:04:40,Not latest view,Serie
12993,12993,Chona,2015-01-08 19:20:47,15,Lost: Temporada 1: Nacida para correr (Capítul...,Apple TV 3 (revised),00:00:14,Not latest view,Serie
12994,12994,Chona,2015-01-08 19:18:32,37,Lost: Temporada 1: Éxodo: Parte 1 (Capítulo 23),Apple TV 3 (revised),00:00:37,Not latest view,Serie
12995,12995,Chona,2015-01-08 19:16:45,18,Lost: Temporada 1: Éxodo: Parte 1 (Capítulo 23),Apple TV 3 (revised),00:00:18,Not latest view,Serie
